---
---
<center>

**<font color='#FF8C00'>.•.•●•.•●⬤●•. Буду рад вашей поддержке! .•●⬤●•.•●•.•.</font>**

<a href="https://www.donationalerts.com/r/politrees" title="Перейти к Donationalerts">
   <img src="https://upload.wikimedia.org/wikipedia/ru/a/ad/DA_Logo_Color.svg" width="200" alt="Donationalerts">
</a>

**Делаю модели на заказ. Подробности в [Telegram](https://t.me/Politrees2)**

---
---

**Будьте в курсе всех обновлений и новостей! Подписывайтесь на мой [Telegram-канал](https://t.me/politrees)**

---

## **<big><<< ТРЕНИРОВКА МОДЕЛИ**

In [ ]:
#@title <big>⬇️ **<font color="red">ШАГ 1:</font> Установка RVC**

REPO_BRANCH = "PolTrain"
ROOT_DIR = "/content/PolTrain"

# ===== ИМПОРТ БИБЛИОТЕК ===== #
import os
from ipywidgets import Button
from google.colab import drive
from torch.cuda import is_available
from IPython.display import clear_output

# ===== ПРОВЕРКА ДОСТУПНОСТИ GPU ===== #
print("Проверка доступности GPU...")
if not is_available():
    raise Exception(
        "\033[91m GPU недоступен! \033[0m\n"
        "К сожалению, у вас нет доступа к GPU на вашем текущем аккаунте. "
        "Пожалуйста, перейдите на другой Google аккаунт, который имеет доступ к GPU, или подождите 24 часа, прежде чем повторить попытку."
        )
print("\033[92m GPU доступен! \033[0m")

# ===== ПОДКЛЮЧЕНИЕ GOOGLE DRIVE И СОЗДАНИЕ ПАПКИ dataset ===== #
if not os.path.isdir('/content/drive'):
    print("\nПодключение к Google Drive...")
    drive.mount('/content/drive')
    !rm -r /content/sample_data
if not os.path.exists('/content/dataset'):
    os.makedirs('/content/dataset')

# ===== КЛОНИРОВАНИЕ РЕПОЗИТОРИЯ ===== #
print("\nКлонирование репозитория...")
!git clone https://github.com/Politrees/PolTrain -b {REPO_BRANCH} {ROOT_DIR}
%cd {ROOT_DIR}
clear_output()

# ===== ВЫВОД СООБЩЕНИЙ ОБ УСТАНОВКЕ ===== #
print("Установка может занять до 5 минут. Пожалуйста, подождите...")
print("По любым вопросам, пишите в Telegram: https://t.me/+GMTP7hZqY0E4OGRi")

# ===== УСТАНОВКА ЗАВИСИМОСТЕЙ И ПАКЕТОВ ===== #
print("\n[1/2] - Установка зависимостей и пакетов...")
!pip install --no-cache-dir uv -q
!uv pip install --no-cache-dir -q "numpy<2" \
                                  "faiss-cpu" \
                                  "torch==2.8.0" \
                                  "torchaudio==2.8.0" \
                                  "torchvision==0.23.0" \
                                  "https://github.com/Politrees/PolTrain/releases/download/fixed-packages/fairseq-0.12.3-cp312-cp312-linux_x86_64.whl"
!sudo apt install -y aria2 > /dev/null 2>&1

# ===== УСТАНОВКА НЕОБХОДИМЫХ МОДЕЛЕЙ ===== #
print("[2/2] - Установка необходимых моделей...")
!python download_files.py

clear_output()
Button(description="\u2714 Готово", button_style="success")

In [ ]:
#@title <big> ⛏️ **<font color="red">ШАГ 2:</font> Обработка данных**

ROOT_DIR = "/content/PolTrain"
SAVE_DIR = "/content/drive/MyDrive/PolTrain"

import os
import re
from ipywidgets import Button
from IPython.display import clear_output

%cd {ROOT_DIR}
clear_output()

#@markdown <details>
#@markdown <summary><b><u><big>Подробнее</big></u></b></summary>
#@markdown
#@markdown > <b><u>model_name</u></b> - **Имя модели**. Разрешено использовать только латиницу, цифры и нижнее подчёркивание ( _ ). Пробелы и специальные символы запрещены. Данное название будет использовано для создания папки с файлами данной модели в <i>`TrainingModel/logs`</i> и в имени .pth файлов в <i>`TrainingModel/assets/weights`</i>. Для продолжения тренировки необходимо будет указать именно это имя.
#@markdown >
#@markdown > **Рекомендация**: Имя модели должно быть уникальным для каждой новой модели, чтобы избежать конфликтов.
#@markdown
#@markdown > <b><u>dataset_folder</u></b> - **Путь к папке с датасетом**. В указанной папке должен находиться аудиофайл (или несколько аудиофайлов) в формате .wav или .flac, на котором будет обучаться модель. Чтобы указать собственный путь, необходимо в панели слева ("Файлы") кликнуть ПКМ по нужной папке и выбрать пункт "Скопировать путь", после чего вставить.
#@markdown >
#@markdown > **Рекомендация**: Используйте датасет длиной не менее 10 минут с чистым голосом без шумов, эха и длительных пауз.
#@markdown
#@markdown > <b><u>sample_rate</u></b> - **Частота дискретизации** - то, насколько детально звук разложен по времени (как FPS у видео). Чем выше - тем качественнее:
#@markdown > * 32.000 Гц (низкая частота, быстрая обработка, менее качественный звук).
#@markdown > * 40.000 Гц (стандартный выбор для большинства задач, хорошее соотношение скорости и качества).
#@markdown > * 48.000 Гц (высокая частота, более качественный звук, требует больше ресурсов).
#@markdown >
#@markdown > **Рекомендация**: Выставляйте частоту не выше, чем у датасета.
#@markdown
#@markdown > <b><u>f0_method</u></b> - **Метод извлечения тона** - то, как будет определяться высота звука (F0) в датасете. Выбор влияет на точность и скорость обработки:
#@markdown > * harvest - традиционный алгоритм, даёт хорошие результаты на чистых вокальных записях, но может ошибаться в сложных условиях (шум, резонансы), медленный.
#@markdown > * rmvpe - продвинутый алгоритм, лучше справляется с шумами и наложенными гармониками, чем harvest, быстрый.
#@markdown > * rmvpe+ - оригинальная улучшенная версия rmvpe, экспериментальная.
#@markdown >
#@markdown > **Рекомендация**: Используйте rmvpe для большинства задач.
#@markdown
#@markdown > <b><u>index_algorithm</u></b> - **Алгоритм извлечения индекса** - то, как будут группироваться данные для обучения модели RVC. Не влияет на качество модели:
#@markdown > * Auto - автоматический выбор в зависимости от размера вашего датасета.
#@markdown > * Faiss - лучше для маленьких и средних датасетов.
#@markdown > * KMeans - лучше для больших датасетов.
#@markdown >
#@markdown > **Рекомендация**: Используйте Auto для автоматического выбора алгоритма.

#@markdown ---
#@markdown * **Имя модели:**
model_name = '' # @param {"type":"string","placeholder":"Дайте имя своей модели"}
#@markdown * **Путь к папке с датасетом:**
dataset_folder = '/content/dataset' #@param {type:"string"}
#@markdown ---
#@markdown * **Частота дискретизации:**
sample_rate = "40000"  # @param ["32000", "40000", "48000"]
#@markdown * **Метод извлечения тона:**
f0_method = "rmvpe"  # @param ["rmvpe", "rmvpe+"]
#@markdown * **Архитектура Fairseq:**
arch_fairseq = "Fairseq"  # @param ["Fairseq", "Fairseq2"]
#@markdown * **Нормализация данных:**
normalize = True # @param {"type":"boolean"}
#@markdown ---
#@markdown * **Генерация индексного файла:**
create_index = True # @param {"type":"boolean"}
#@markdown * **Алгоритм извлечения индекса:**
index_algorithm = "Auto"  # @param ["Auto", "Faiss", "KMeans"]
#@markdown ---

# ========================================================================== #
# ДОПОЛНИТЕЛЬНЫЕ ПАРАМЕТРЫ | Скрыты, чтоб незнающие люди ничего не поломали.

# Максимальная длина фрагментов / По умолчанию: 3с.000мс.
percentage = 3.0


# ================================================== #


# ===== ПРОВЕРКА НА ПРАВИЛЬНОСТЬ ВВОДА ИМЕНИ МОДЕЛИ ===== #
if not re.match(r"^[a-zA-Z0-9_]+$", model_name):
    raise ValueError(f"Имя '{model_name}' содержит недопустимые символы!")
else:
    os.makedirs(f'{SAVE_DIR}/{model_name}', exist_ok=True)

# ===== ПРОВЕРКА НА СУЩЕСТВОВАНИЕ ПАПКИ С ДАТАСЕТОМ ===== #
if not os.path.exists(dataset_folder):
    raise FileNotFoundError(f"Папка '{dataset_folder}' не существует!")
if not os.listdir(dataset_folder):
    raise FileNotFoundError(f"Папка '{dataset_folder}' пуста!")

# ==================================================== #
# ======= ОБРАБОТКА ДАННЫХ И ГЕНЕРАЦИЯ ИНДЕКСА ======= #

# Сегментирование и ресемплинг датасета
preprocess_script = f"{ROOT_DIR}/rvc/train/preprocess/preprocess.py"
!python {preprocess_script} {SAVE_DIR}/{model_name} {dataset_folder} {percentage} {sample_rate} {normalize}
clear_output()

# Извлечение среднего тона и характеристик звука
preparing_data_script = f"{ROOT_DIR}/rvc/train/preprocess/preparing_data.py"
!python {preparing_data_script} {SAVE_DIR}/{model_name} {arch_fairseq} {f0_method} {sample_rate} 2
clear_output()

# Генерация индексного файла на основе характеристик
extract_index_script = f"{ROOT_DIR}/rvc/train/extract/extract_index.py"
if create_index:
    !python {extract_index_script} {SAVE_DIR}/{model_name} {index_algorithm}


In [ ]:
#@title <big>🤖 **<font color="red">ШАГ 3:</font> Тренировка модели**

ROOT_DIR = "/content/PolTrain"
SAVE_DIR = "/content/drive/MyDrive/PolTrain"

import os
import traceback
from urllib.parse import urlparse
from IPython.display import clear_output
from subprocess import PIPE, STDOUT, Popen

%cd {ROOT_DIR}

#@markdown <details>
#@markdown <summary><b><u><big>Подробнее</big></u></b></summary>
#@markdown
#@markdown > <b><u>epochs</u></b> - **Общее количество эпох** - эпоха, до которой будет продолжаться тренировка модели, после чего остановится. По завершению тренировки происходит сохранение .pth файла в `TrainingModel/assets/weights` с названием `[имя модели].pth`. Если тренировка оборвётся из-за лимитов Google Colab, то продолжить её можно будет с последней сохранённой эпохи.
#@markdown >
#@markdown > **Среднее рекомендуемое количество**: 500-1000
#@markdown
#@markdown > <b><u>save_epoch</u></b> - **Частота сохранения** - интервал эпох, с которым будет происходить сохранение тренировки. Сопровождается сохранением .pth файла в `TrainingModel/assets/weights` с названием по шаблону: `[имя модели]_e[номер эпохи]_s[количество шагов].pth`
#@markdown
#@markdown > <b><u>pretrain</u></b> - **Предварительно обученная модель** - модель, заранее обученная на большом количестве специфичных данных для сохранения определённых закономерностей. Используется для улучшения звучания тренируемых моделей в зависимости от конкретных задач:
#@markdown > * **Default** - стандартная модель, хорошо передаёт произношение и мощность голоса из датасета, но требовательна к датасету.
#@markdown > * **Snowie v3.1** - модель для улучшения русскоязычного акцента, мощность голоса слегка мягче, хорошо справляется с маленькими датасетами.
#@markdown > * **TITAN-Medium** - модель для улучшения англоязычного акцента. `[недостаточно информации]`
#@markdown > * **KLM v4.3 x3** - модель для улучшения корееязычного акцента, очень похожа по звучанию с Default. `[недостаточно информации]`
#@markdown >
#@markdown > **Рекомендуется выбирать исходя из своей задачи**
#@markdown
#@markdown > <b><u>custom_pretrained</u></b> - **Пользовательская предварительно обученная модель** - раздел для самостоятельной загрузки модели, отсутствующей в списке. `Для опытных пользователей.`
#@markdown
#@markdown > <b><u>batch_size</u></b> - **Размер пакета** - количество фрагментов датасета, обрабатываемых за один шаг. Влияет на стабильность, скорость тренировки и использование вычислительных мощностей. Чем выше - тем лучше, но не на каждом датасете Google Colab выдержит слишком высокие значения - требуется корректировать от конкретного случая, либо включать экономию памяти (следующий пункт).
#@markdown >
#@markdown > **Рекомендуемое значение пишется по завершению обработки датасета**

#@markdown ---
#@markdown * **Общее количество эпох:** <small>`(1-10000)`
epochs = "500" # @param {type:"string"}
#@markdown * **Частота сохранения моделей и чекпоинтов:** <small>`(1-100)`
save_epoch = "25" # @param {type:"string"}
#@markdown * **Сохранять бэкап чекпоинта:** <small>`требуется 3гб свободного места`</small>
save_backup = False # @param {type:"boolean"}
#@markdown ---
# #@markdown * **Генератор:**
vocoder = "HiFi-GAN"
#@markdown * **Предварительно обученная модель:**
pretrain = "Default" # @param ["Без претрейна", "Default", "Snowie v3.1", "TITAN-Medium", "KLM v4.3 x3"]
#@markdown * **Пользовательская предварительно обученная модель:**
custom_pretrained = False # @param {type:"boolean"}
d_pretrained_link = "" # @param {"type":"string","placeholder":"Ссылка на D файл"}
g_pretrained_link = "" # @param {"type":"string","placeholder":"Ссылка на G файл"}
#@markdown > Поддерживаются только ссылки с [HuggingFace](https://huggingface.co/Politrees/RVC_resources/tree/main/pretrained/v2)
#@markdown ---
#@markdown * **Размер пакета:** <small>`(2-16)`
batch_size = 8  # @param {type:"slider", min:2, max:16, step:2}
#@markdown * **Запаковать модель в ZIP-архив:**
save_to_zip = False # @param {type:"boolean"}
#@markdown ---
#@markdown * **Включить TensorBoard:** » <small>Руководство по **[TensorBoard](https://github.com/Bebra777228/TrainVocModel-EN/tree/DOCS/docs/tensorboard)**
tensorboard = True # @param {type:"boolean"}

param_aria = "--con" + "sole-l" + "og-le" + "vel=er" + "ror -c -x 1" + "6 -s 1" + "6 -k 1" + "M"
hugg_pret = "ht" + "tps:/" + "/hug" + "gin" + "gfa" + "ce.co" + "/Poli" + "tree" + "s/RV" + "C_res" + "ourc" + "es/re" + "solv" + "e/ma" + "in/pret" + "rain" + "ed/v2"

pretrain_outpath = f"{ROOT_DIR}/rvc/models/pretraineds"
!rm -r {pretrain_outpath}  > /dev/null 2>&1

clear_output()

sample_rate_k = f"{int(int(sample_rate) / 1000)}k"
models = {
    "Default": [
        (f"{sample_rate_k}/Default/f0D{sample_rate_k}.pth", f"default_D.pth"),
        (f"{sample_rate_k}/Default/f0G{sample_rate_k}.pth", f"default_G.pth"),
    ],
    "Snowie v3.1": [
        (f"{sample_rate_k}/Snowie/D_SnowieV3.1_{sample_rate_k}.pth", f"Snowie_D.pth"),
        (f"{sample_rate_k}/Snowie/G_SnowieV3.1_{sample_rate_k}.pth", f"Snowie_G.pth"),
    ],
    "TITAN-Medium": [
        (f"{sample_rate_k}/TITAN/D-f0{sample_rate_k}-TITAN-Medium.pth", f"TITAN_Medium_D.pth"),
        (f"{sample_rate_k}/TITAN/G-f0{sample_rate_k}-TITAN-Medium.pth", f"TITAN_Medium_G.pth"),
    ],
    "KLM v4.3 x3": [
        (f"{sample_rate_k}/KLM/D_KLM43_X3_{sample_rate_k}.pth", f"KLM_D.pth"),
        (f"{sample_rate_k}/KLM/G_KLM43_X3_{sample_rate_k}.pth", f"KLM_G.pth"),
    ],
    "Без претрейна": [(None, None), (None, None)],
}

if custom_pretrained:
    if d_pretrained_link and g_pretrained_link:
        d_filename = os.path.basename(urlparse(d_pretrained_link).path)
        g_filename = os.path.basename(urlparse(g_pretrained_link).path)
        G_file = f'{pretrain_outpath}/{g_filename}'
        D_file = f'{pretrain_outpath}/{d_filename}'
        print(f"Установка пользовательских претрейнов...\nG_file - {g_filename}\nD_file - {d_filename}")
        !aria2c {param_aria} {g_pretrained_link} -d {pretrain_outpath} -o {g_filename} > /dev/null 2>&1
        !aria2c {param_aria} {d_pretrained_link} -d {pretrain_outpath} -o {d_filename} > /dev/null 2>&1
    else:
        raise ValueError("Для custom_pretrained необходимо указать ссылки на D и G файлы претрейна.")
elif pretrain == "Без претрейна":
    print(f"Внимание! Тренировка без претрейна может быть более долгой и требовательной к датасету.")
    G_file = None
    D_file = None
else:
    print(f"Установка претрейна {pretrain}...")
    for f in models[pretrain]:
        if f[0] is not None:
            !aria2c {param_aria} {hugg_pret}/{f[0]} -d {pretrain_outpath} -o {f[1]} > /dev/null 2>&1

    G_file = f'{pretrain_outpath}/{models[pretrain][1][1]}' if models[pretrain][1][1] else None
    D_file = f'{pretrain_outpath}/{models[pretrain][0][1]}' if models[pretrain][0][1] else None

def click_train(
    experiment_dir,
    model_name,
    save_epoch_interval,
    total_epochs,
    batch_size,
    sample_rate,
    vocoder,
    pretrained_G,
    pretrained_D,
    save_backup,
    save_to_zip,
):
    print("\nЗапуск программы...")

    cmd = (
        f'python {ROOT_DIR}/rvc/train/train.py '
        f'--experiment_dir "{experiment_dir}" '
        f'--model_name "{model_name}" '
        f'--batch_size {batch_size} '
        f'--sample_rate {sample_rate} '
        f'--total_epoch {total_epochs} '
        f'--save_every_epoch {save_epoch_interval} '
        f'--save_to_zip {save_to_zip} '
        f'--vocoder "{vocoder}" '
        f'--save_backup {save_backup} '
        f'{"--pretrain_g %s" % pretrained_G if pretrained_G is not None else ""} '
        f'{"--pretrain_d %s" % pretrained_D if pretrained_D is not None else ""}'
    )

    try:
        p = Popen(
            cmd,
            bufsize=1,
            text=True,
            shell=True,
            stdout=PIPE,
            stderr=STDOUT,
            cwd=os.getcwd(),
            universal_newlines=True,
        )

        for line in p.stdout:
            line = line.strip()
            if not any(unwanted in line for unwanted in [
                "All log messages before absl::InitializeLog()",
                "Unable to register cuDNN factory",
                "Unable to register cuBLAS factory",
                "computation placer already registered"
            ]):
                print(line)

        p.wait()

    except Exception as e:
        with open(f"{SAVE_DIR}/{model_name}/error_log.txt", "w") as f:
            f.write("Произошла ошибка:\n")
            f.write(traceback.format_exc())
        raise Exception(f"Произошла ошибка: {e}")

    return "Программа закрыта."

if tensorboard:
    %load_ext tensorboard
    %tensorboard --logdir {SAVE_DIR}
training_log = click_train(
    SAVE_DIR,
    model_name,
    save_epoch,
    epochs,
    batch_size,
    sample_rate,
    vocoder,
    G_file,
    D_file,
    save_backup,
    save_to_zip,
)
print(training_log)

## **<big><<< ПРОДОЛЖИТЬ ТРЕНИРОВКУ МОДЕЛИ**

In [ ]:
#@title <big>⬇️ **<font color="red">ШАГ 1:</font> Установка RVC**

REPO_BRANCH = "PolTrain"
ROOT_DIR = "/content/PolTrain"

# ===== ИМПОРТ БИБЛИОТЕК ===== #
import os
from ipywidgets import Button
from google.colab import drive
from torch.cuda import is_available
from IPython.display import clear_output

# ===== ПРОВЕРКА ДОСТУПНОСТИ GPU ===== #
print("Проверка доступности GPU...")
if not is_available():
    raise Exception(
        "\033[91m GPU недоступен! \033[0m\n"
        "К сожалению, у вас нет доступа к GPU на вашем текущем аккаунте. "
        "Пожалуйста, перейдите на другой Google аккаунт, который имеет доступ к GPU, или подождите 24 часа, прежде чем повторить попытку."
        )
print("\033[92m GPU доступен! \033[0m")

# ===== ПОДКЛЮЧЕНИЕ GOOGLE DRIVE И СОЗДАНИЕ ПАПКИ dataset ===== #
if not os.path.isdir('/content/drive'):
    print("\nПодключение к Google Drive...")
    drive.mount('/content/drive')
    !rm -r /content/sample_data

# ===== КЛОНИРОВАНИЕ РЕПОЗИТОРИЯ ===== #
print("\nКлонирование репозитория...")
!git clone https://github.com/Politrees/PolTrain -b {REPO_BRANCH} {ROOT_DIR}
%cd {ROOT_DIR}

clear_output()
Button(description="\u2714 Готово", button_style="success")

In [ ]:
#@title <big>🤖 **<font color="red">ШАГ 2:</font> Тренировка модели**

ROOT_DIR = "/content/PolTrain"
SAVE_DIR = "/content/drive/MyDrive/PolTrain"

import os
import traceback
from IPython.display import clear_output
from subprocess import PIPE, STDOUT, Popen

%cd {ROOT_DIR}

#@markdown <details>
#@markdown <summary><b><u><big>Подробнее</big></u></b></summary>
#@markdown
#@markdown > <b><u>model_name</u></b> - **Имя модели** - название модели, которую требуется дотренировать. Название должно соответствовать названию папки модели в `TrainingModel/logs`.
#@markdown
#@markdown > <b><u>epochs</u></b> - **Общее количество эпох** - эпоха, до которой будет продолжаться тренировка модели, после чего остановится. По завершению тренировки происходит сохранение .pth файла в `TrainingModel/assets/weights` с названием `[имя модели].pth`. Если тренировка оборвётся из-за лимитов Google Colab, то продолжить её можно будет с последней сохранённой эпохи.
#@markdown
#@markdown > <b><u>save_epoch</u></b> - **Частота сохранения** - интервал эпох, с которым будет происходить сохранение тренировки. Сопровождается сохранением .pth файла в `TrainingModel/assets/weights` с названием по шаблону: `[имя модели]_e[номер эпохи]_s[количество шагов].pth`.
#@markdown
#@markdown > <b><u>sample_rate</u></b> - **Частота дискретизации** - значение, выбранное при создании модели.
#@markdown
#@markdown > <b><u>batch_size</u></b> - **Размер пакета** - значение, выбранное при создании модели, либо корректируется в зависимости от конкретного случая.

#@markdown ---
#@markdown * **Имя модели:**
model_name = '' # @param {"type":"string","placeholder":"Введите имя своей модели"}
#@markdown ---
#@markdown * **Общее количество эпох:** <small>`(1-10000)`
epochs = "1000" # @param {type:"string"}
#@markdown * **Частота сохранения моделей и чекпоинтов:** <small>`(1-100)`
save_epoch = "25" # @param {type:"string"}
#@markdown * **Сохранять бэкап чекпоинта:** <small>`требуется 3гб свободного места`</small>
save_backup = False # @param {type:"boolean"}
#@markdown ---
#@markdown * **Размер пакета:** <small>`(2-16)`
batch_size = 8  # @param {type:"slider", min:2, max:16, step:2}
#@markdown * **Запаковать модель в ZIP-архив:**
save_to_zip = False # @param {type:"boolean"}
#@markdown ---
#@markdown * **Включить TensorBoard:** » <small>Руководство по **[TensorBoard](https://github.com/Bebra777228/TrainVocModel-EN/tree/DOCS/docs/tensorboard)**
tensorboard = True # @param {type:"boolean"}


clear_output()

def click_train(
    experiment_dir,
    model_name,
    save_epoch_interval,
    total_epochs,
    batch_size,
    save_backup,
    save_to_zip,
):
    print("\nЗапуск программы...")

    cmd = (
        f'python {ROOT_DIR}/rvc/train/train.py '
        f'--experiment_dir "{experiment_dir}" '
        f'--model_name "{model_name}" '
        f'--batch_size {batch_size} '
        f'--total_epoch {total_epochs} '
        f'--save_every_epoch {save_epoch_interval} '
        f'--save_backup {save_backup} '
        f'--save_to_zip {save_to_zip} '
    )

    try:
        p = Popen(
            cmd,
            bufsize=1,
            text=True,
            shell=True,
            stdout=PIPE,
            stderr=STDOUT,
            cwd=os.getcwd(),
            universal_newlines=True,
        )

        for line in p.stdout:
            line = line.strip()
            if not any(unwanted in line for unwanted in [
                "All log messages before absl::InitializeLog()",
                "Unable to register cuDNN factory",
                "Unable to register cuBLAS factory",
                "computation placer already registered"
            ]):
                print(line)

        p.wait()

    except Exception as e:
        with open(f"{SAVE_DIR}/{model_name}/error_log.txt", "w") as f:
            f.write("Произошла ошибка:\n")
            f.write(traceback.format_exc())
        raise Exception(f"Произошла ошибка: {e}")

    return "Программа закрыта."

if tensorboard:
    %load_ext tensorboard
    %tensorboard --logdir {SAVE_DIR} --port=8888
training_log = click_train(
    SAVE_DIR,
    model_name,
    save_epoch,
    epochs,
    batch_size,
    save_backup,
    save_to_zip,
)
print(training_log)

## <big>
---
    Все файлы, созданные в процессе тренировки,
    автоматически сохраняются на Google Диск в папку PolTrain.

    * Путь к .pth файлу:
       PolTrain / [Имя Модели] / [Имя Модели].pth                      - Финальная модель
       PolTrain / [Имя Модели] / weights / [Имя Модели]_e10_s500.pth   - Промежуточные модели
    * Путь к .index файлу:
       PolTrain / [Имя Модели] / [Имя Модели].index                    - Индексный файл

    Также, если вы включите параметр "save_to_zip",
    то по окончании тренировки будет создан ZIP-архив,
    в котором будут содержатся: финальная модель и индексный файл.
    
    * Путь к .zip файлу:
       PolTrain / [Имя Модели] / [Имя Модели].zip                      - ZIP-архив
---

<center><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small>.

**[<big><big><big> Pol-Litrees RVC </big></big></big>](https://colab.research.google.com/drive/1W39tbdYxR1NSVNHG6EDRiKkY4JM0f60B)**

**CoverGen — многофункциональный интерфейс для создания каверов.**

**PolGen — интерфейс для преобразования одного голоса в другой и текста в речь.**

<small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small><small>

---

### Количество посещений:

<img src="https://counter.seku.su/cmoe?name=PolTrain_&theme=mbs" /><br>